In [74]:
import elasticsearch
import csv
from elasticsearch import Elasticsearch
from collections import defaultdict
import sys
import csv
import random
import gzip
import os
from collections import defaultdict
import pandas as pd

from nltk import sent_tokenize, word_tokenize
import nltk
#nltk.download('punkt')
from pycontractions import Contractions
from nltk.stem import WordNetLemmatizer 
from nltk.stem import LancasterStemmer
from nltk.stem import PorterStemmer
import gensim 
from gensim.models import Word2Vec 
import gensim.downloader
from scipy import spatial
import numpy as np
from gensim.models.doc2vec import Doc2Vec,  TaggedDocument
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True,limit=1000000)  

In [4]:
df_querys = pd.read_csv("orcas-doctrain-queries.tsv",encoding="utf8",delimiter="\t",header=None)
df_querys.rename( columns ={0: 'id', 1: 'query'}, inplace = True )
df_querys = df_querys.astype('str')
df_querys.set_index('id',inplace = True)
df_querys.head()

,query
id,
9265503,github
6832981,youtube
9571352,!
4896888,! c
10001890,! c++


In [5]:
df_qrels = pd.read_csv("orcas-doctrain-qrels.tsv",encoding="utf8",delimiter="\t",header=None) 
df_qrels.rename( columns ={0: 'test'}, inplace = True )
df_qrels = pd.DataFrame( df_qrels.test.str.split(' ',3).tolist(),columns = ['query_id','i1','doc_id','i2'])
df_qrels.set_index(['query_id','doc_id'],inplace = True)
df_qrels.drop(['i1', 'i2'], axis=1,inplace = True)
df_qrels.head()

Empty DataFrame
Columns: []
Index: [(9265503, D1265400), (9265503, D3438005), (6832981, D2923232), (9571352, D1238374), (9571352, D889000)]

In [6]:
df_top100 = pd.read_csv("orcas-doctrain-top100.gz",encoding="utf8",delimiter="\t",header=None,nrows= 100000)
df_top100.rename( columns ={0: 'test'}, inplace = True )
df_top100 = pd.DataFrame( df_top100.test.str.split(' ',4).tolist(),columns = ['query_id','i1','doc_id','i2','i3'])
df_top100.set_index(['query_id'],inplace = True)
#df_qrels.drop(['i1', 'i2'], axis=1,inplace = True)
df_top100.head()

,i1,doc_id,i2,i3
query_id,,,,
2000000,Q0,D278797,1,-4.75082 IndriQueryLikelihood
2000000,Q0,D692485,2,-4.9127 IndriQueryLikelihood
2000000,Q0,D1769574,3,-4.92883 IndriQueryLikelihood
2000000,Q0,D519555,4,-4.96246 IndriQueryLikelihood
2000000,Q0,D3200438,5,-4.98945 IndriQueryLikelihood


In [7]:
df_querys_small = df_querys.loc[df_top100.index.unique()]
df_querys_small.head()

,query
query_id,
2000000,it support model
2000001,brian game of thrones
2000003,free fill dirt
2000004,bank of america official website
2000005,bushel n peck


In [8]:
docoffset = {}
with gzip.open("msmarco-docs-lookup.tsv.gz", 'rt', encoding='utf8') as f:
    tsvreader = csv.reader(f, delimiter="\t")
    for [docid, _, offset] in tsvreader:
        docoffset[docid] = int(offset)
        
def getcontent(docid, f):
    """getcontent(docid, f) will get content for a given docid (a string) from filehandle f.
    The content has four tab-separated strings: docid, url, title, body.
    """

    f.seek(docoffset[docid])
    line = f.readline()
    assert line.startswith(docid + "\t"), \
        f"Looking for {docid}, found {line}"
    return line#.rstrip()        
        
        
def get_docs(doc_list,docs ="msmarco-docs.tsv"):
    
    df = pd.DataFrame(columns = ['id','url','title','body'])
    df.set_index('id',inplace = True)
    
    with open(docs, encoding="utf8") as f:    
        for doc in doc_list:       
            i = getcontent(doc, f).split('\t')
            row = pd.Series({'url':i[1],'title':i[2],'body':i[3]}, name = i[0])
            df = df.append(row)
    
    return df
            
def get_ranking(query_id,dataframe = df_top100):
    return dataframe.loc[query_id]['doc_id'].tolist()        
        
get_docs(get_ranking('2000001')).head()        

,url,title,body
id,,,
D1826672,https://en.wikipedia.org/wiki/Iron_Throne_%28A...,Iron Throne ( A Song of Ice and Fire),"""From Wikipedia, the free encyclopedianavigati..."
D1343166,https://en.wikipedia.org/wiki/Game_of_Thrones_...,Game of Thrones (season 1),"""From Wikipedia, the free encyclopedia (Redire..."
D2530279,https://en.wikipedia.org/wiki/Game_of_Thrones_...,Game of Thrones (season 1),"""From Wikipedia, the free encyclopedianavigati..."
D138178,https://en.wikipedia.org/wiki/List_of_Game_of_...,List of Game of Thrones episodes,"""From Wikipedia, the free encyclopedianavigati..."
D1644363,https://en.wikipedia.org/wiki/Game_of_Thrones_...,Game of Thrones (2014 video game),"""This article is about the 2014 graphic advent..."


In [10]:
df_triples = pd.read_csv("triples.tsv",encoding="utf8",delimiter="\t",header=None)
df1 = df_triples[[0,1,3,4,5]]
df1.insert(0,'y',1)
df0 = df_triples[[0,1,7,8,9]]
df0.insert(0,'y',0)
df0 = df0.rename( columns ={7: 3, 8: 4,9:5} )

df_triples = pd.concat([df1, df0])
df_triples = df_triples.rename(   columns ={0: 'query_id', 1: 'query',3:'url',4:'title',5:'body'}          )

df_triples['url'] = df_triples.apply(lambda x: clean_text(clean_url(x['url'])) ,axis=1)
df_triples['query'] = df_triples.apply(lambda x: clean_text(x['query']) ,axis=1)
df_triples['title'] = df_triples.apply(lambda x: clean_text(x['title']) ,axis=1)
df_triples['body'] = df_triples.apply(lambda x: clean_text(x['body']) ,axis=1)
df_triples.head()

,y,query_id,query,url,title,body
0,1,2000000,it support model,wikiped org wik techn support,techn support,"from wikiped , the fre encyclopedianavig searc..."
1,1,2000001,bri gam of thrones,wikiped org wik bran stark,bran stark,"from wikiped , the fre encyclopedianavig searc..."
2,1,2000003,fre fil dirt,dirtfil,.,`` i found a sit nee fil dirt on dirt fill.com...
3,1,2000004,bank of americ off websit,loc bankofameric,find a fin cent or atm,find a fin cent or atmarizon arkansa californ ...
4,1,2000005,bushel n peck,bushelnpeckdel,nut and delicy produc at afford pric ...,nut and delicy produc at afford pric ... the b...


In [9]:

p=LancasterStemmer()

def clean_url(url):
        url = url.replace('com','').replace('en','').split('/')
 
        return ' '.join([ word.replace('.',' ').replace('_',' ') +' ' for word in url[1:]]).strip().lower() 

def clean_text(text):
        
        sentences = sent_tokenize(str(text).lower())
        cleaned_text = ' '.join([ ' '.join([ p.stem(word) for word in  word_tokenize(sentence) ]) for sentence in sentences ])
        return cleaned_text 
    
clean_url('https://en.wikipedia.org/wiki/Technical_support')    

'wikipedia org  wiki  technical support'

In [ ]:
es = Elasticsearch()
es.info()

In [ ]:
if es.indices.exists(INDEX_NAME):
    es.indices.delete(INDEX_NAME)

In [ ]:
INDEX_NAME ='ms_macro_index'


INDEX_SETTINGS = {
    'mappings': {
            'properties': {
                'url': {
                    'type': 'text',
                    'term_vector': 'yes',
                    'analyzer': 'english'
                },
                'title': {
                    'type': 'text',
                    'term_vector': 'yes',
                    'analyzer': 'english'
                },
                'body': {
                    'type': 'text',
                    'term_vector': 'yes',
                    'analyzer': 'english'
                }
            }
        }
    }

es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)

In [ ]:
def index_all(querys):
    
    for query in querys:
        print(query)
        docs = get_ranking(query)
        index_docs(docs)
  
def index_docs(list_docIds,docs ="msmarco-docs.tsv" ):
    
        
    with open(docs, encoding="utf8") as f:    
        for doc in list_docIds:       
            i = getcontent(doc, f).split('\t')
            if not es.exists(index=INDEX_NAME, id=doc):
                es.index(index=INDEX_NAME, id=i[0], body={'url':i[1],'title':i[2],'body':i[3]})       

In [30]:
tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(data)]

In [11]:
data = []
for i,row in df_triples.iterrows():
    for sent in sent_tokenize(row['body']):
        sentence =[]
        for word in word_tokenize(sent):
            sentence.append(word)
        
        data.append(sentence)
print(len(data))
model = gensim.models.Word2Vec(data, min_count = 1, size = 100, window = 5) 

201354


In [32]:
model_doc  = Doc2Vec(tagged_data, vector_size=20, window=2, min_count=1, workers=10, epochs = 1)

In [ ]:
def w2v(a,b):
    
    vec_a = np.mean([model.wv.word_vec(word) for word in a if word in model.wv.vocab], axis=0) 
    vec_b = np.mean([model.wv.word_vec(word) for word in b if word in model.wv.vocab], axis=0) 
    
    return 1 - spatial.distance.cosine(vec_a, vec_b)



In [136]:
test = df_triples[1900:1905].copy()

In [137]:
def w2v(a,b):
    
    vec_a = np.mean([model.wv.word_vec(word) for word in a if word in model.wv.vocab], axis=0) 
    vec_b = np.mean([model.wv.word_vec(word) for word in b if word in model.wv.vocab], axis=0) 
    
    return 1 - spatial.distance.cosine(vec_a, vec_b)
    
    

In [138]:
test['w2v_url'] = test.apply(lambda x: w2v(x['query'],x['title']) ,axis=1)

test

C:\Users\einar\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\einar\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.
C:\Users\einar\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\einar\anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,y,query_id,query,url,title,body,w2v_url
900,0,2000987,medit and anxy,www ergy-healing-info best-crystal-for-healing...,best cryst for heal anxy,best cryst for heal anxietyth best cryst and s...,0.926961
901,0,2000988,how to expand bracket,www teacherscho au mathem how-to libr htm,topalgebr,[ teach ' cho softw hom pag ] stuck on yo home...,0.967504
902,0,2000989,dfa civil pay custom serv,www dfa mil car careerpath technicians-callcte...,nan,techn & cal cent repres dfa employ individ in ...,0.583577
903,0,2000990,what is on the cw tonight,www tvguide tvshows supergirl 694533,supergirl,overview new photo episod video clip tv list c...,0.902352
904,0,2000991,apostroph exampl sent,www grammarbook gramm pronoun asp,.,"pronoun definit a pronoun ( i , me , he , she ...",NaN


In [41]:
def avg_sentence_vector(words, model, num_features):
    #function to average all words vectors in a given paragraph
    featureVec = np.zeros((num_features,), dtype="float32")
    n = 0

    for word in words:
        if word in model.wv.vocab:
            n = n+1
            featureVec = np.add(featureVec, model[word])

    if n>0:
        featureVec = np.divide(featureVec, n)
    return featureVec


In [125]:
 a = avg_sentence_vector('technician', model, 300)

C:\Users\einar\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys


In [126]:
b= np.mean([model.wv.word_vec(word) for word in 'electric' if word in model.wv.vocab], axis=0) 

C:\Users\einar\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [127]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(a.reshape(1, -1),b.reshape(1, -1))

array([[0.94877696]], dtype=float32)

In [128]:
1 - spatial.distance.cosine(a, b)

0.9487770795822144